In [2]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa

--2019-03-08 16:51:19--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 52.85.107.203, 52.85.107.15, 52.85.107.230, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|52.85.107.203|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5528 (5.4K) [application/octet-stream]
Saving to: ‘phix.fa.3’

phix.fa.3           100%[===================>]   5.40K  --.-KB/s    in 0s      

2019-03-08 16:51:19 (12.2 MB/s) - ‘phix.fa.3’ saved [5528/5528]



In [17]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [18]:
genome = readGenome('phix.fa')

In [19]:
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences
# this will yield a list of the positions where a match occurs in the template t

In [20]:
t = 'AGCTTAGATAGC'
p = 'AG'
naive(p, t)

[0, 5, 9]

In [21]:
import random
def generateReads(genome, numReads, readLen):
    ''' Generate reads from random positions in the given genome. '''
    reads = []
    for _ in range(numReads):
        start = random.randint(0, len(genome)-readLen) - 1
        reads.append(genome[start : start+readLen])
    return reads

In [22]:
# Generate 100 reads of length 100
reads = generateReads(genome, 100, 100)

# Count how many reads match the genome exactly
numMatched = 0
for r in reads:
    matches = naive(r, genome)
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, len(reads)))

100 / 100 reads matched the genome exactly!


In [9]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq

--2019-03-08 16:51:19--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 52.85.107.203, 52.85.107.15, 52.85.107.230, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|52.85.107.203|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 254384 (248K) [audio/mpeg]
Saving to: ‘ERR266411_1.first1000.fastq.2’

ERR266411_1.first10 100%[===================>] 248.42K  --.-KB/s    in 0.07s   

2019-03-08 16:51:20 (3.65 MB/s) - ‘ERR266411_1.first1000.fastq.2’ saved [254384/254384]



In [23]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline() # skip name line
            seq = fh.readline().rstrip() # read base sequence
            fh.readline() # skip placeholder line
            qual = fh.readline().rstrip() #base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities


In [24]:
phix_reads = readFastq('ERR266411_1.first1000.fastq')

In [25]:
numMatched = 0
n = 0
for r in phix_reads:    # loop the list of reads that were read in as phix_reads
    matches = naive(r, genome)    # use naive match function to test if the reads align to the phix genome
    n += 1
    if len(matches) > 0:   # is there at least 1 match for a given read (there could be more than one?)
        numMatched += 1    # tally the number of aligned reads
print('%d / %d reads matched the genome exactly!' % (numMatched, n))
# only 7/1000 reads matched exactly - because of sequencing error? may be differences between the sequenced organism and the template
# maybe we should just take a part of the read instead?

0 / 2 reads matched the genome exactly!


In [26]:
# Now let's try matching just the first 30 bases of each read
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]  # just taking the first 30 bases
    matches = naive(r, genome)
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))
# this improved the matching, but it's still less than 50% - doesn't take into account forward vs. reverse!

0 / 2 reads matched the genome exactly!


In [27]:
# Paste in the reverse complement function from 1.02
def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

In [29]:
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]  # just taking the first 30 bases
    matches = naive(r, genome)
    matches.extend(naive(reverseComplement(r), genome)) # this adds any matches that occurred on the reverse complement
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))
# much better!

KeyError: 'TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTC'

1. Question 1

How many times does AGGT or its reverse complement ACCT occur in the lambda virus genome? E.g. if AGGT occurs 10 times and ACCT occurs 12 times, you should report 22.

In [30]:
lamda = readGenome('lambda_virus.fa')

In [31]:
# create a new naive matching algorithm that deals with reverse complement
def new_naive(p, t):
    r = reverseComplement(p)
    if r == p:
        return naive(p,t)
    else:
        return naive(p,t) + naive (r,t)
# this will yield a list of the positions where a match occurs in the template t

In [32]:
match = new_naive("AGGT", lamda)
len(match)


306

In [33]:
forward = naive("AGGT",lamda)
len(forward)

150

In [34]:
reverse = naive("ACCT",lamda)
len(reverse)

156

In [35]:
# naive exact matching algorithm that is strand-aware
# Instead of looking only for occurrences of P in T, additionally look for occurrences of the reverse complement of P in T. 
def naive_with_rc(p, t):
    r = reverseComplement(p)
    if r == p:
        return naive(p,t)
    else:
        return naive(p,t) + naive(r,t)

In [36]:
occurrences = new_naive('AGGT', lamda)
print('AGGT in lambda_virus:')
print('# occurrences: %d' % len(occurrences))

AGGT in lambda_virus:
# occurrences: 306


2. Question 2

How many times does TTAA or its reverse complement occur in the lambda virus genome?

Hint: TTAA and its reverse complement are equal, so remember not to double count.

In [37]:
occurrences = new_naive('TTAA', lamda)
print('TTAA in lambda_virus:')
print('# occurrences: %d' % len(occurrences))

TTAA in lambda_virus:
# occurrences: 195


3. Question 3

What is the offset of the leftmost occurrence of ACTAAGT or its reverse complement in the Lambda virus genome? E.g. if the leftmost occurrence of ACTAAGT is at offset 40 (0-based) and the leftmost occurrence of its reverse complement ACTTAGT  is at offset 29, then report 29.

In [38]:
print('The left-most occurrence of ACTAAGT in lambda_virus: %d' % min(new_naive('ACTAAGT',lamda)))


The left-most occurrence of ACTAAGT in lambda_virus: 26028


4. Question 4

What is the offset of the leftmost occurrence of AGTCGA or its reverse complement in the Lambda virus genome?

In [39]:
print('The left-most occurrence of AGTCGA in lambda_virus: %d' % min(new_naive('AGTCGA',lamda)))

The left-most occurrence of AGTCGA in lambda_virus: 450


5. Question 5

As we will discuss, sometimes we would like to find approximate matches for P in T. That is, we want to find occurrences with one or more differences.

For Questions 5 and 6, make a new version of the naive function called naive_2mm that allows up to 2 mismatches per occurrence. Unlike for the previous questions, do not consider the reverse complement here. We're looking for approximate matches for P itself, not its reverse complement.

￼

For example, ACTTTA occurs twice in ACTTACTTGATAAAGT, once at offset 0 with 2 mismatches, and once at offset 4 with 1 mismatch. So naive_2mm('ACTTTA', 'ACTTACTTGATAAAGT') should return the list [0, 4]

Hint: See this notebook for a few examples you can use to test your naive_2mm function.

How many times does TTCAAGCC occur in the Lambda virus genome when allowing up to 2 mismatches? 


WRONG!

In [40]:
def naive_2mm(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):
        mismatch = 0
        match = True
        for j in range(len(p)):
            if t[i+j] != p[j]:
                mismatch += 1
                if mismatch > 2:
                    match = False
                    break
        if match:
            occurrences.append(i)
    return occurrences

In [41]:
occurrences_5 = naive_2mm('TTCAAGCC', lamda)

print('TTCAAGCC in lambda_virus:')
print('# occurrences: %d' % len(occurrences_5))

TTCAAGCC in lambda_virus:
# occurrences: 191


6. Question 6

What is the offset of the leftmost occurrence of AGGAGGTT
in the Lambda virus genome when allowing up to 2 mismatches?


WRONG!

In [42]:
print('The left-most occurrence of AGGAGGTT in lambda_virus: %d' % min(naive_2mm('AGGAGGTT',lamda)))

The left-most occurrence of AGGAGGTT in lambda_virus: 49


Question 7

Finally, download and parse the provided FASTQ file containing real DNA sequencing reads derived from a human:

https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR037900_1.first1000.fastq

Note that the file has many reads in it and you should examine all of them together when answering this question. The reads are taken from this study:

Ajay, S. S., Parker, S. C., Abaan, H. O., Fajardo, K. V. F., & Margulies, E. H. (2011). Accurate

and comprehensive sequencing of personal genomes. Genome research, 21(9), 1498-1505.

This dataset has something wrong with it; one of the sequencing cycles is poor quality.

Report which sequencing cycle has the problem. Remember that a sequencing cycle corresponds to a particular offset in all the reads. For example, if the leftmost read position seems to have a problem consistently across reads, report 0. If the fourth position from the left has the problem, report 3. Do whatever analysis you think is needed to identify the bad cycle. It might help to review the "Analyzing reads by position" video.





WRONG!

In [ ]:
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR037900_1.first1000.fastq

In [43]:
seqs, quals = readFastq('ERR037900_1.first1000.fastq')

In [44]:
def phred33ToQ(qual):
    return ord(qual) - 33

In [45]:
def createHist(qualities):
    # Create a histogram of quality scores
    hist = [0]*50
    for qual in qualities:
        for phred in qual:
            q = phred33ToQ(phred)
            hist[q] += 1
    return hist
h = createHist(quals)
print(h)


[0, 0, 17723, 0, 2, 11, 11, 28, 23, 55, 100, 111, 86, 174, 185, 272, 317, 259, 390, 1523, 2782, 762, 286, 413, 403, 538, 351, 694, 971, 777, 1024, 1449, 1341, 1312, 1916, 2233, 3025, 4043, 6640, 45696, 2074, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [46]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(range(len(h)), h)
plt.show()

ModuleNotFoundError: No module named 'matplotlib'